# Tarea 2: Naive Bayes, Linear Models y Neural Networks
**Procesamiento de Lenguaje Natural (CC6205-1 - Otoño 2024)**

## Tarjeta de identificación

**Nombres:** ```Martín Bravo, Felipe Fierro```

**Fecha límite de entrega 📆:** 30/04.

**Tiempo estimado de dedicación:** 4 horas


## Instrucciones
Bienvenid@s a la segunda tarea en el curso de Natural Language Processing (NLP). Esta tarea tiene como objetivo evaluar los contenidos teóricos de las últimas semanas de clases posteriores a la tarea 1, enfocado principalmente en **Naive Bayes**, **Linear Models** y **Neural Networks**. Si aún no has visto las clases, se recomienda visitar los links de las referencias.

La tarea consta de una una parte práctica con el fín de introducirlos a la programación en Python enfocada en NLP.

* La tarea es en **grupo** (maximo hasta 3 personas).
* La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
* El formato de entrega es este mismo Jupyter Notebook.
* Al momento de la revisión su código será ejecutado. Por favor verifiquen que su entrega no tenga errores de compilación.
* Completar la tarjeta de identificación. Sin ella no podrá tener nota.

> **Importante:** Esta tarea tiene varios resultados experimentales que pueden variar de acuerdo a sus propias implementaciones. No se busca que los resultados sean exactamente los mismos (por ejemplo, que el accuracy fue el mismo que el que esta en la tarea). Lo importante es que implementen sus funciones, las sepan explicar y que puedan hacer varios experimentos.

## Material de referencia

Diapositivas del curso 📄
    
- [Naive Bayes](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-NB.pdf)
- [Linear Models](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-linear.pdf)
- [Neural Networks](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-neural.pdf)

Videos del curso 📺

- Naive Bayes: [Parte 1](https://www.youtube.com/watch?v=kG9BK9Oy1hU), [Parte 2](https://www.youtube.com/watch?v=Iqte5kKHvzE), [Parte 3](https://www.youtube.com/watch?v=TSJg0_X3Abk)

- Linear Models: [Parte 1](https://www.youtube.com/watch?v=zhBxDsNLZEA), [Parte 2](https://www.youtube.com/watch?v=Fooua_uaWSE), [Parte 3](https://www.youtube.com/watch?v=DqbzhdQa1eQ), [Parte 4](https://www.youtube.com/watch?v=1nfWWXqfAzA)

- Neural Networks: [Parte 1](https://www.youtube.com/watch?v=oHZHA8h2xN0), [Parte 2](https://www.youtube.com/watch?v=2lXank0W6G4), [Parte 3](https://www.youtube.com/watch?v=BUDIi9qItzY), [Parte 4](https://www.youtube.com/watch?v=KKN2Ipy-vGk)


## P0. Cargar un dataset

Importamos algunas librerias que seran utiles.

In [100]:
import pandas as pd
from collections import namedtuple

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/martinbravodiaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Inicializamos el dataset con particiones de entrenamiento y test. Es un dataset de clasificacion multi-clase de oraciones. Cada oracion puede tener una unica etiqueta ?, + o -. Donde ? indica que la oracion es una pregunta, - que la oracion es negativa y + positiva.

In [101]:
document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

raw_train_set = [
              ['Do you have plenty of time?', '?'],
              ['Does she have enough money?','?'],
              ['Did they have any useful advice?','?'],
              ['What day is today?','?'],
              ["I don't have much time",'-'],
              ["She doesn't have any money",'-'],
              ["They didn't have any advice to offer",'-'],
              ['Have you plenty of time?','?'],
              ['Has she enough money?','?'],
              ['Had they any useful advice?','?'],
              ["I haven't much time",'-'],
              ["She hasn't any money",'-'],
              ["He hadn't any advice to offer",'-'],
              ['How are you?','?'],
              ['How do you make questions in English?','?'],
              ['How long have you lived here?','?'],
              ['How often do you go to the cinema?','?'],
              ['How much is this dress?','?'],
              ['How old are you?','?'],
              ['How many people came to the meeting?','?'],
              ['I’m from France','+'],
              ['I come from the UK','+'],
              ['My phone number is 61709832145','+'],
              ['I work as a tour guide for a local tour company','+'],
              ['I’m not dating anyone','-'],
              ['I live with my wife and children','+'],
              ['I often do morning exercises at 6am','+'],
              ['I run everyday','+'],
              ['She walks very slowly','+'],
              ['They eat a lot of meat daily','+'],
              ['We were in France that day', '+'],
              ['He speaks very fast', '+'],
              ['They told us they came back early', '+'],
              ["I told her I'll be there", '+']
]
tokenized_train_set = [document(words=tuple(word_tokenize(d[0].lower())), class_=d[1]) for d in raw_train_set]
train_set = pd.DataFrame(data=tokenized_train_set)

raw_test_set = [
             ['Do you know who lives here?','?'],
             ['What time is it?','?'],
             ['Can you tell me where she comes from?','?'],
             ['How are you?','?'],
             ['I fill good today', '+'],
             ['There is a lot of history here','+'],
             ['I love programming','+'],
             ['He told us not to make so much noise','+'],
             ['We were asked not to park in front of the house','+'],
             ["I don't have much time",'-'],
             ["She doesn't have any money",'-'],
             ["They didn't have any advice to offer",'-'],
             ['I am not really sure','+']
]
tokenized_test_set = [document(words=tuple(word_tokenize(d[0].lower())), class_=d[1]) for d in raw_test_set]
test_set = pd.DataFrame(data=tokenized_test_set)

Separar en X e y, donde X son oraciones tokenizadas e y es la clase a predecir (o target).

In [102]:
X_train, y_train = train_set.drop(columns="class_"), train_set["class_"]
pd.concat([X_train, y_train], axis=1).sample(10)

,words,class_
32,"(they, told, us, they, came, back, early)",+
5,"(she, does, n't, have, any, money)",-
21,"(i, come, from, the, uk)",+
9,"(had, they, any, useful, advice, ?)",?
1,"(does, she, have, enough, money, ?)",?
18,"(how, old, are, you, ?)",?
3,"(what, day, is, today, ?)",?
31,"(he, speaks, very, fast)",+
13,"(how, are, you, ?)",?
11,"(she, has, n't, any, money)",-


Cantidad de oraciones por clase:

In [103]:
train_set.groupby("class_").count()

,words
class_,
+,13
-,7
?,14


(X, y) para el conjunto de test:

In [104]:
X_test, y_test = test_set.drop(columns="class_"), test_set["class_"]
pd.concat([X_test, y_test], axis=1).sample(10)

,words,class_
5,"(there, is, a, lot, of, history, here)",+
9,"(i, do, n't, have, much, time)",-
6,"(i, love, programming)",+
4,"(i, fill, good, today)",+
2,"(can, you, tell, me, where, she, comes, from, ?)",?
10,"(she, does, n't, have, any, money)",-
3,"(how, are, you, ?)",?
1,"(what, time, is, it, ?)",?
0,"(do, you, know, who, lives, here, ?)",?
8,"(we, were, asked, not, to, park, in, front, of...",+


Cantidad de oraciones por clase en el conjunto de test:

In [105]:
test_set.groupby("class_").count()

,words
class_,
+,6
-,3
?,4


**Importante:** Hasta el momento hemos creado nuestros conjuntos de train y test. A continuacion ustedes deben implementar tres modelos de clasificacion: Naive-bayes, Linear Model y Neural Network. Aqui va un resumen de cada pregunta y lo que se les pide implementar:

* P1: Naive-bayes
 - Implementar `fit` y `predict`
 - Entrenar
 - Evaluar

* P2: Linear Model
 - Implementar `fit` con *on-line gradient descent* y `predict`
 - Entrenar
 - Evaluar

* P3: Neural Network
 - Implementar un iterador de datos con `datasets` y `dataloaders`
 - Implementar una red neuronal con `pytorch`
 - Implementar loop de entrenamiento de una NN
 - Entrenar
 - Evaluar

## P1. Implementar y evaluar Multinomial Naive-Bayes (2 puntos)

### Clase para clasificador

Cree una clase MyMultinomialNB que en su inicializador reciba el parámetro alpha para su clasficador.

Además, debe implementar los métodos `fit(X, y)`y `predict(X)`.

```python
class MyMultinomialNB():
  def __init__(self, alpha, ...):
    ...

  def fit(self, X, y):
    ...
  
  def predict(self, X):
    ...
    return prediction
```
Para computar el entrenamiento de nuestro clasificador debemos:
- extraer el vocabulario,
- determinar las probabilidades $p(c_j)$ para cada una de las clases posibles,
- determinar las probabilidades $p(w_i|c_j)$ para cada una de las palabras y cada una de las clases.

Para lograr lo anterior, también deberán implementar el método `predict_proba(X)`:

```python
  def predict_proba(self, X):
    return prob
```

**Underflow prevention:** En vez de hacer muchas multiplicaciones de `float`s, reemplácenlas por sumas de logaritmos para prevenir errores de precisión. (Revisen la diapo 26 de las slides).

En su implementación deben considerar la tecnica de *Laplace Smoothing* vista en clases. Especificamente considere que su clase `MyMultinomialNB` reciba un parámetro `alpha` no negativo (es decir, mayor o igual a cero). De tal forma que el la probabilidad de una palabra $w$ dado la clase $c$ viene dado por lo siguiente:

$$
p_\alpha (w|c) = \frac{\#(w, c) + \alpha}{N + \alpha |V|}
$$

donde $\alpha$ es el parámetro `alpha` de *Laplace Smoothing*. Mientras que los otras notaciones corresponden a

* $\#(w, c)$ numero de veces que ocurre la palabra $w$ en documentos con la clase $c$ (pensar en un gran documento $D_c$ que concatena todos los documentos de clase $c$ y luego calcula la frecuencia de la palabra $w$ en $D_c$),
* $N$ es igual a $\sum \{\#(w', c): w' \in V\}$ donde $V$ es el vocabulario,
* $|V|$ tamaño del vocabulario.

### Implementación (1.5 pts.)

Escriba aquí la implementación de la clase `MyMultinomialNB`.

In [106]:
import numpy as np

a = np.array([1,2,2,3,3,1,1])

print(tuple(word_tokenize("hola como estás")))

('hola', 'como', 'estás')


In [107]:
import numpy as np

class MyMultinomialNB():
    def __init__(self, alpha=1.0):
        self.alpha = alpha  # Parámetro de suavizado Laplaciano

    def fit(self, X, y):
        # Conteo de clases y vocabulario
        self.classes, class_counts = np.unique(y, return_counts=True)
        self.class_counts = dict(zip(self.classes, class_counts))
        self.vocab = set()
        self.class_word_counts = {c: {} for c in self.classes}

        # Conteo de palabras por clase
        for c in self.classes:
            mask = (y == c)
            documents = X[mask] # Documentos de la clase c
            word_counts = {}
            for document in documents.values.flatten():
                for word in document:
                    self.vocab.add(word)
                    word_counts[word] = word_counts.get(word, 0) + 1
            self.class_word_counts[c] = word_counts

        # Calcula la probabilidad de cada clase
        total_documents = len(y)
        self.class_probs = {c: count / total_documents for c, count in self.class_counts.items()}

        # Calcula la probabilidad condicional de cada palabra dado cada clase
        self.word_probs = {}
        for c in self.classes:
            total_words_in_class = sum(self.class_word_counts[c].values())
            self.word_probs[c] = {}
            for word in self.vocab:
                word_count = self.class_word_counts[c].get(word, 0)
                self.word_probs[c][word] = (word_count + self.alpha) / (total_words_in_class + self.alpha * len(self.vocab))

    def predict(self, X):
        pred = []
        for document in X.values.flatten():
            max_prob = -np.inf
            predicted_class = None
            for c in self.classes:
                log_prob = np.log(self.class_probs[c])
                for word in document:
                    if word in self.vocab:
                        log_prob += np.log(self.word_probs[c].get(word, 1e-10))  # Evitar log(0)
                if log_prob > max_prob:
                    max_prob = log_prob
                    predicted_class = c
            pred.append(predicted_class)
        return pred


### Entrenamiento (0.2 pts.)
A continuación, inicialicen y entrenen (ajusten) su clasificador con los datos de entrenamiento.

In [108]:
nb_model = MyMultinomialNB(alpha=0.1)
nb_model.fit(X_train, y_train);

Pruébenlo utilizando el método `predict()` que implementaron.

In [109]:
from sklearn.metrics import classification_report

In [110]:
# Predict train-set
y_pred = nb_model.predict(X_train)
print(y_pred)

['?', '?', '?', '?', '-', '-', '-', '?', '?', '?', '-', '-', '-', '?', '?', '?', '?', '?', '?', '?', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+']


In [111]:
# Metricas en el conjunto de train
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           +       1.00      1.00      1.00        13
           -       1.00      1.00      1.00         7
           ?       1.00      1.00      1.00        14

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando el método `predict`.


In [112]:
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           +       1.00      0.67      0.80         6
           -       0.60      1.00      0.75         3
           ?       1.00      1.00      1.00         4

    accuracy                           0.85        13
   macro avg       0.87      0.89      0.85        13
weighted avg       0.91      0.85      0.85        13



Comenten sus resultados. Estudien que ocurre para alpha=0, 1 y L donde L es un numero muy grande.

```
Comentar aquí.
```

## P2. Implementar y evaluar Linear Models (2 puntos)

### Clase para clasificador

Cree una clase MyLinearModel para su clasficador. Debe implementar los métodos `fit(X, y, learning_rate, epochs)`y `predict(X)`.

```python
class MyLinearModel():
  def __init__(self, ...):
    ...

  def fit(self, X, y, learning_rate, epochs):
    ...
  
  def predict(self, X):
    ...
    return prediction
```

El modelo lineal que debe implementar viene dado por:
$$
\vec{\hat{y}} = \text{softmax}(\vec{x} \cdot W + \vec{b})\\
\vec{\hat{y}}_{[i]} = \frac{\exp{z_i}}{\sum_{j} \exp{z_j}}\\
z_i = \vec{x} \cdot W_{[:, i]} + \vec{b}_{[i]}
$$
donde $\vec{x}$ es un documento representado con bolsas de palabras (BoW), $W$ es la matriz de pesos y $\vec{b}$ el bias.

El modelo linea debe ajustarlo considerando como objetivo minimizar la cross-entropy loss, es decir:

$$
L_\text{cross-entropy}(\vec{\hat{y}}, \vec{y}) = - \sum_i \vec{y}_{[i]} \log{ \left( \vec{\hat{y}}_{[i]} \right) }
$$

Para representar un documento `(i, am, not, really, sure)` vectorialmente, utilice `CountVectorizer` de sklearn. De esta manera, el documento queda representado como sigue:

|    |   i |   he |   am |   are |   not |   yes |   really |   sure |
|---:|----:|-----:|-----:|------:|------:|------:|---------:|-------:|
|  0 |   1 |    0 |    1 |     0 |     1 |     0 |        1 |      1 |

**Observación:** Si el documento repite palabras entonces tendrá un número mayor a 1. Si el documento no tiene la palabra entonces tiene un 0. Pensar que las palabras `(he, are, yes)` provienen de otros documentos. Recuerde que el `CountVectorizer` se entrena con más de un documento (es decir, un corpus). Aquí debe usar el conjunto de train.

El método `fit(X, y, learning_rate, epochs)` debe ajustar un `CountVectorizer` para representar vectorialmente el documento. Debe guardar el `CountVectorizer` para cuando quiera hacer predicciones. Dentro del método `fit(X, y, learning_rate, epochs)` debe implementar *On-line gradient descent* (visto en clases), es decir, descenso de gradiente usando un data-point por iteración. Su método debe ser capaz de recibir un `learning_rate` para ponderar el gradiente en cada iteración y fijar un número de `epochs`. Luego de entrenar debe guardar los pesos de su modelo lineal, es decir, $(W, \vec{b})$.

En el algoritmo de descenso de gradiente usando un data-point por iteración, o *On-line gradient descent*, debe implementar manualmente las derivadas. Como conoce el modelo lineal y la funcion objetivo, entonces puede calcular manualmente las derivadas. Para ejemplificar, en cada paso del algoritmo de optimizacion debe actualizar los pesos $(W, \vec{b})$ del siguiente modo:

$$
W \leftarrow W - \lambda \nabla_{W} L_\text{cross-entropy}\\
\vec{b} \leftarrow \vec{b} - \lambda \nabla_{\vec{b}} L_\text{cross-entropy}\\
$$
donde $\lambda$ es el parámetro `learning_rate`, $\nabla_{W} L_\text{cross-entropy}$ el gradiente de la Loss con repecto a la matriz de pesos $W$ y $\nabla_{\vec{b}} L_\text{cross-entropy}$ para el bias $\vec{b}$.

Para implementar el algoritmo *On-line gradient descent* les recomendamos (no es obligatorio hacerlo de este modo) definir una función `get_derivative_W(x, y_target, y_pred, n_classes)` que calcule $\nabla_{W} L_\text{cross-entropy}$ y lo mismo con una función `get_derivative_b(y_target, y_pred, n_classes)` que calcule $\nabla_{\vec{b}} L_\text{cross-entropy}$.

Para implementar el método `predict(self, X)` debera usar su `CountVectorizer` definido en `fit(X, y, learning_rate, epochs)` para representar del mismo modo cualquier documento tanto en train como en test.

### Implementación (1.5 pts.)
Implemente un modelo lineal con métodos `fit(X, y)` y `predict(X)`

$$
L_\text{cross-entropy}(\vec{\hat{y}}, \vec{y}) = - \sum_i \vec{y}_{[i]} \log{ \left( \vec{\hat{y}}_{[i]} \right) }
$$

$$ 
= - \sum_i \frac{\exp{z_i}}{\sum_{j} \exp{z_j}} \log{ \left( \frac{\exp{z_i}}{\sum_{j} \exp{z_j}} \right) }
$$

$$
= - \sum_i \frac{\exp{\vec{x} \cdot W_{[:, i]} + \vec{b}_{[i]}}}{\sum_{j} \exp{\vec{x} \cdot W_{[:, j]} + \vec{b}_{[j]}}} \log{ \left( \frac{\exp{\vec{x} \cdot W_{[:, i]} + \vec{b}_{[i]}}}{\sum_{j} \exp{\vec{x} \cdot W_{[:, j]} + \vec{b}_{[j]}}} \right) }
$$

$$
\nabla_W L = - \sum_i \frac{\exp{z_i}}{\sum_{j} \exp{z_j}} \nabla_W \log{ \left( \frac{\exp{z_i}}{\sum_{j} \exp{z_j}} \right) }
$$

In [113]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

class MyLinearModel():
    def __init__(self):
        self.vectorizer = CountVectorizer()
        self.W = None
        self.b = None

    def fit(self, X, y, learning_rate, epochs, verbose=False):

        # Pasar los documentos a frases para poder vectorizarlos
        X = X["words"].apply(lambda x: " ".join(x)).values

        # Pasar los documentos a una representación vectorial
        X = self.vectorizer.fit_transform(X).toarray().T

        # Pasar los valores de predicción a números
        y = y.apply(lambda x: 0 if x == '-' else 1 if x == '+' else 2).values

        # Inicializar pesos
        self.W = np.random.randn(3, X.shape[0])
        self.b = np.random.randn(3)

        # Repetimos el proceso epoch cantidad de veces
        for epoch in range(epochs):

            # Para cada documento
            for x_i, it in zip(X.T, y):

                # One-hot encoding
                y_i = np.zeros(3)
                y_i[it] = 1

                # Calculamos la predicción
                y_i_pred = softmax(self.W @ x_i + self.b)

                # Calculamos la función de pérdida cross-entropy
                loss = -np.sum(y_i * np.log(y_i_pred))

                # Actualizamos los pesos
                self.W -= learning_rate * np.outer(y_i_pred - y_i, x_i)
                self.b -= learning_rate * (y_i_pred - y_i)

            if verbose:
                print(f"Epoch {epoch}, Loss: {loss}")
            
        return 
    
    def predict(self, X):
        # Pasar los documentos a frases para poder vectorizarlos
        X = X["words"].apply(lambda x: " ".join(x)).values

        # Pasar los documentos a una representación vectorial
        X = self.vectorizer.transform(X).toarray().T

        preds = []
        # Realizar la predicción
        for x_i in X.T:

            # Calculamos la predicción
            y_i_pred = softmax(self.W @ x_i + self.b)

            # Obtenemos la mayor probabilidad
            it = np.argmax(y_i_pred)
        
            # Pasar los valores de predicción a letras
            pred = '+' if it == 1 else '-' if it == 0 else '?'
            preds.append(pred)
            
        return preds


### Entrenamiento (0.2 pts.)
Inicialicen y entrenen su clasificador con los datos de entrenamiento.

In [114]:
linear_model = MyLinearModel()
linear_model.fit(
    X_train, y_train,
    learning_rate=0.02,
    epochs=15,
    verbose=True)

Epoch 0, Loss: 0.3957748723023839
Epoch 1, Loss: 0.30757547291453147
Epoch 2, Loss: 0.24743353935685344
Epoch 3, Loss: 0.2049797007740419
Epoch 4, Loss: 0.1740520568874803
Epoch 5, Loss: 0.15087688710416158
Epoch 6, Loss: 0.1330561469598279
Epoch 7, Loss: 0.11901966388586889
Epoch 8, Loss: 0.10771686448787511
Epoch 9, Loss: 0.09843289068895368
Epoch 10, Loss: 0.09067386014641597
Epoch 11, Loss: 0.08409336889317488
Epoch 12, Loss: 0.07844469029498186
Epoch 13, Loss: 0.07354930306568618
Epoch 14, Loss: 0.06927586118445113


Pruébenlo utilizando el método `predict()` que implementaron.

In [115]:
# Predict train-set
y_pred = linear_model.predict(X_train)

In [116]:
# Metricas en el conjunto de train
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           +       0.83      0.77      0.80        13
           -       0.75      0.86      0.80         7
           ?       0.79      0.79      0.79        14

    accuracy                           0.79        34
   macro avg       0.79      0.80      0.80        34
weighted avg       0.80      0.79      0.79        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando el método `predict`.

In [117]:
y_pred = linear_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           +       1.00      0.17      0.29         6
           -       0.50      0.67      0.57         3
           ?       0.50      1.00      0.67         4

    accuracy                           0.54        13
   macro avg       0.67      0.61      0.51        13
weighted avg       0.73      0.54      0.47        13



Comenten sus resultados. Estudien que ocurre para al menos tres combinaciones de learning rates y epochs, por ejemplo `learning_rate, epochs = (0.02, 15), (0.1, 10), (0.005, 30)`.

```
Comentar aquí.
```

In [118]:
combs = [(0.1, 10), (0.1, 30), (0.01, 15), (0.001, 15)]

for lr, ep in combs:
    linear_model = MyLinearModel()
    linear_model.fit(
        X_train, y_train,
        learning_rate=lr,
        epochs=ep,
        verbose=False)

    y_pred = linear_model.predict(X_test)
    print(f"Learning rate: {lr}, Epochs: {ep}")
    print(classification_report(y_test, y_pred))
    print("")


Learning rate: 0.1, Epochs: 10
              precision    recall  f1-score   support

           +       0.75      0.50      0.60         6
           -       0.60      1.00      0.75         3
           ?       0.50      0.50      0.50         4

    accuracy                           0.62        13
   macro avg       0.62      0.67      0.62        13
weighted avg       0.64      0.62      0.60        13


Learning rate: 0.1, Epochs: 30
              precision    recall  f1-score   support

           +       0.83      0.83      0.83         6
           -       0.75      1.00      0.86         3
           ?       1.00      0.75      0.86         4

    accuracy                           0.85        13
   macro avg       0.86      0.86      0.85        13
weighted avg       0.87      0.85      0.85        13


Learning rate: 0.01, Epochs: 15
              precision    recall  f1-score   support

           +       0.67      0.67      0.67         6
           -       0.67      0.67

Podemos notar que a medida que disminuimos el learning rate, el modelo disminuye su precisión. Por otro lado, a medida que aumentamos el número de epochs, el modelo mejora su precisión.

## P3. Implementar y evaluar Neural Networks (2 puntos)

### Especificaciones del clasificador

<img src="https://docs.google.com/drawings/d/e/2PACX-1vSXJm5I61m6w0RHTwBL-iMyeFLr2wXBrKNYxdU8Bu1ymuCFPD9dAPsCzPfvIqwSr8uCiYvWMdnGy1if/pub?w=818&h=503" >

En esta última pregunta, ustedes deberánimplementar y evaluar redes neuronales (como la de la figura de arriba). Para esto debera implementar tres secciones principales:

1. Sección iterador,
2. Sección modelo, y
3. Sección loop de entrenamiento.

> **Recomendación:** Para completar esta pregunta puede guiarse del Auxiliar 2 (clase del día 18/04).

*Seccion iterador*

Para ayudarnos a con el entrenamiento y testing, vamos a utilizar las clases `Dataset` y `DataLoader` de `pytorch` ([ver documentación](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)). En esta sección deberá implementar un contenedor para su conjunto de datos usando la clase `Dataset` de `pytorch`. Para esto deberá crear su propia clase `MyDataset` para gestionar los datos. Ésto le permitirá iterar sobre el conjunto mediante el iterador `DataLoader` de `pytorch` y entrenar sin hacer ningún pre-procesamiento extra a los datos.

**Observación:** Si considera por funcionalidad cambiar los parámetros de la clase `MyDataset` puede hacerlo. Asimismo, puede definir otros parámetros para los métodos de su clase.


```python
class MyDataset(Dataset):
    def __init__(self, data, bow_cols):
      ...

    def __len__(self):
      ...

    def __getitem__(self, index):
      ...
      return x_bow, label
```

*Sección modelo*

En esta sección deberán implementar la clase `MyNeuralNetwork` del modulo de `pytorch` llamado `nn.Module` con el proposito de diseñar una red neuronal como la figura de arriba. Para mas detalle sobre las redes ver Clase NLP-Neural.pdf Slide número 8.

**Observación:** La figura de arriba es solo ilustrativa, ustedes pueden variar la dimension input y output de la capa oculta. Sin embargo deben mantener fija la dimension de la entrada y salida de la red. La entrada depende del tamaño del vocabulario. Mientras que la salida depende de la cantidad de clases de su problema de clasificación (en nuestro caso igual a 3).

Es importante que la clase `MyNeuralNetwork` tenga implementadas apropiadamente el `__init__` con las dimensiones y el `forward` con entrada tipo BoW retornando el último estado de la red (output layer). En el `forward` recomendamos utilizar funciones de activación tipo `nn.ReLU`. Sin embargo, no es completamente obligatorio por lo que pueden usar otras.

```python
class MyNeuralNetwork(nn.Module):
    def __init__(self,
                 dim_vocab,
                 num_classes,
                 dim_hidden_input,
                 dim_hidden_output):

        super(MyNeuralNetwork, self).__init__()
        torch.manual_seed(42)
      ...

    def forward(self, xs_bow):
      ...
      return last_state
  ```

*Sección loop de entrenamiento*

En esta sección deberán implementar el loop de entrenamiento de su red neuronal. Para esto, primero deben definir un `criterion`, en nuestro caso `nn.CrossEntropyLoss()` con la libreria de `pytorch`. Sucesivamente debera definir un optimizador, en nuestro caso `optim.SGD` desde el modulo `optim` de `pytorch`.

El loop de entrenamiento debe seguir la siguiente estructura:
```python
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:
    ...
```

donde `train_loader` proviene del iterador generado en la "sección iterador".

Dentro de "doble for" debera conjugar apropiadamente `opti.zero_grad()`, `loss = criterion(...)`, `loss.backward()` y `opti.step()` con tal de entrenar correctamente su red neuronal. Incluso entrenar, ya que a veces si no se hace de forma correcta entonces tristemente ¡su red no entrena!

> **Recomendación:** Puede guiarse del Auxiliar 2 para implementar el loop de entrenamiento.

### Preparación de la GPU y los datos de train/test

Importar la libreria `pytorch` y `numpy`

In [119]:
import torch
import numpy as np

Verificar que esta usando GPU. Sino, dirígase a **Runtime > Change runtime type** y seleccione la opción **T4 GPU**.

In [120]:
torch.cuda.is_available()

False

Preparación de los conjuntos train y test

In [121]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(tokenizer=lambda x: list(x), preprocessor=lambda x: x, token_pattern=None)

bow_train = pd.DataFrame(
    bow.fit_transform(train_set["words"]).toarray(),
    columns=bow.get_feature_names_out()
)
bow_test = pd.DataFrame(
    bow.transform(test_set["words"]).toarray(),
    columns=bow.get_feature_names_out()
)

bow_label_train = bow_train.astype(float).copy()
bow_label_test = bow_test.astype(float).copy()

map_from_class_to_int = {
    "?": 0,
    "+": 1,
    "-": 2
}

bow_label_train["class_"] = train_set["class_"]
bow_label_train["int_class_"] = train_set["class_"].apply(lambda x: map_from_class_to_int[x])

bow_label_test["class_"] = test_set["class_"]
bow_label_test["int_class_"] = test_set["class_"].apply(lambda x: map_from_class_to_int[x])

bow_label_train.sample(10)

,'ll,61709832145,6am,?,a,advice,and,any,anyone,are,...,we,were,what,wife,with,work,you,’,class_,int_class_
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,+,1
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,?,0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,+,1
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,2
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,?,0
25,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,+,1
26,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,+,1
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-,2
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,0


### Implementación (1.7 pts.)

#### Iterador de conjunto de datos
Implemente su clase `MyDataset` para acceder al dataset.

In [122]:
from torch.utils.data import Dataset
class MyDataset(Dataset):

    def __init__(self, data, bow_cols):
        self.data = data
        self.bow_cols = bow_cols

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label = int(self.data.loc[index, "int_class_"])
        x_bow = torch.tensor(self.data.loc[index, self.bow_cols]. # Obtenemos el vector x_{index}
        values.astype(float)).to(torch.float32) # y lo convertimos a tensor de float32
        return x_bow, label

Inicializar cada dataloader con sus cotenedor datos para train y test, y número de batches.

In [123]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    MyDataset(data = bow_label_train, bow_cols = bow_train.columns),
    batch_size = 5, num_workers = 1, shuffle=False)

test_loader = DataLoader(
    MyDataset(data = bow_label_test, bow_cols = bow_test.columns),
    batch_size = 5, num_workers = 1, shuffle=False)

Ejemplo de prueba para un batch de entrenamiento

In [124]:
# batch = next(iter(train_loader))
# print( batch )
# print( batch[0].shape, sample[1].shape )

#### Modelo

Implemente a continuación su red neuronal

In [126]:
import torch.nn as nn
class MyNeuralNetwork(nn.Module):
  # Implementar aquí su NN
    def __init__(self,
                 dim_vocab,
                 num_classes,
                 dim_hidden_input,
                 dim_hidden_output):
      """Inicializa la red neuronal

      Returns:
        None
      """
      super(MyNeuralNetwork, self).__init__()
    
      torch.manual_seed(42)

      # Definimos las capas del modelo

      # Primera Capa
      self.first_layer = nn.Linear(dim_vocab, dim_hidden_input)
    
      # Capa Oculta
      self.hidden_layer = nn.Linear(dim_hidden_input, dim_hidden_output)
    
      # Última Capa
      self.last_layer = nn.Linear(dim_hidden_output, num_classes)
    
      # Función de activación
      self.relu = nn.ReLU(inplace=False)
      
    def forward(self, xs_bow):
      """Calcula la ultima capa mediante las capas intermedias de la red

      Args:
        xs_bow: Tensor

      Returns:
        Tensor con los valores de prediccion
      """
      ## Implementar aquí el forward-pass
      
      # Hacemos el forward-pass
      first_state = self.first_layer(xs_bow)
      first_state = self.relu(first_state)

      hidden_state = self.hidden_layer(first_state)
      hidden_state = self.relu(hidden_state)

      last_state = self.last_layer(hidden_state)

      return last_state

Ejemplo de prueba para su modelo NN para un batch de entrenamiento

In [127]:
test = MyNeuralNetwork(
    dim_vocab=len(train_loader.dataset.bow_cols),
    num_classes=3,
    dim_hidden_input=10,
    dim_hidden_output=5).cuda()

batch = next(iter(train_loader))

test(batch[0].cuda())

AssertionError: Torch not compiled with CUDA enabled

#### Entrenamiento

Consideren las siguientes funciones que les serán utiles. Si lo desea puede modificarlas a su conveniencia.

In [ ]:
def get_loss(net, iterator, criterion):
    net.eval()
    total_loss = 0
    num_evals = 0
    with torch.no_grad():
      for xs_bow, labels in iterator:
          xs_bow, labels = xs_bow.cuda(), labels.cuda()

          logits = net(xs_bow)

          loss = criterion(logits, labels)

          total_loss += loss.item() * xs_bow.shape[0]
          num_evals += xs_bow.shape[0]

    return total_loss / num_evals

def get_preds_tests_nn(net, iterator):
  net.eval()
  preds, tests = [], []
  with torch.no_grad():
    for xs_bow, labels in iterator:
      xs_bow, labels = xs_bow.cuda(), labels.cuda()

      logits = net(xs_bow)

      soft_probs = nn.Sigmoid()(logits)

      preds += np.argmax(soft_probs.tolist(), axis=1).tolist()
      tests += labels.tolist()

    return np.array(preds), np.array(tests)

A continuación, inicialicen y entrenen su clasificador con los datos de entrenamiento.

In [ ]:
import torch.optim as optim

params = {
    "dim_vocab": len(train_loader.dataset.bow_cols),
    "num_classes": 3,
    "dim_hidden_input": 5,
    "dim_hidden_output": 5,
    "learning_rate": 0.4,
    "epochs": 15
}

# Inicialice su red neuronal
net = MyNeuralNetwork(
    dim_vocab=params["dim_vocab"],
    num_classes=params["num_classes"],
    dim_hidden_input=params["dim_hidden_input"],
    dim_hidden_output=params["dim_hidden_output"]).cuda()

device = "cuda" if torch.cuda.is_available() else "cpu"

# Definir la Loss = Cross-entropy
criterion = nn.CrossEntropyLoss().cuda()

# Definir el optimizador = SGD: Stochastic-gradient Descent
opti = optim.SGD(net.parameters(), lr = params["learning_rate"])

# Definir el numero de epocas de entrenamiento
epochs = params["epochs"]

## Implementar desde aqui el ciclo de entrenamiento
## para cada epoca en el conjunto de train
for epoch in range(epochs):
  for (xs_bow, labels) in train_loader:

    opti.zero_grad()

    xs_bow, labels = xs_bow.to(device), labels.to(device)

    logits = net(xs_bow)

    loss = criterion(logits, labels)

    loss.backward()

    opti.step()

  total_loss = get_loss(net, train_loader, criterion)
  y_preds, y_tests = get_preds_tests_nn(net, train_loader)
  acc = (y_preds == y_tests).sum() / y_preds.shape[0]

  print("Epoca {} completada! Loss: {} Accuracy: {}".format(epoch, total_loss, acc))

Epoca 0 completada! Loss: 1.1054631629410911 Accuracy: 0.38235294117647056
Epoca 1 completada! Loss: 1.0930101818898146 Accuracy: 0.38235294117647056
Epoca 2 completada! Loss: 1.0800085768980139 Accuracy: 0.38235294117647056
Epoca 3 completada! Loss: 1.056380967006964 Accuracy: 0.38235294117647056
Epoca 4 completada! Loss: 0.9195144391235184 Accuracy: 0.47058823529411764
Epoca 5 completada! Loss: 0.7975653784678263 Accuracy: 0.6176470588235294
Epoca 6 completada! Loss: 0.6394507672418567 Accuracy: 0.7058823529411765
Epoca 7 completada! Loss: 0.4972488123594838 Accuracy: 0.7647058823529411
Epoca 8 completada! Loss: 0.5996851416523842 Accuracy: 0.6176470588235294
Epoca 9 completada! Loss: 0.33634046876036067 Accuracy: 0.8235294117647058
Epoca 10 completada! Loss: 0.2385189199601026 Accuracy: 0.8823529411764706
Epoca 11 completada! Loss: 0.0932115251198411 Accuracy: 1.0
Epoca 12 completada! Loss: 0.05484988332233008 Accuracy: 1.0
Epoca 13 completada! Loss: 0.03735693112727912 Accuracy: 1.

Pruebe su modelo entrenado con la función `get_preds_tests_nn`.

In [ ]:
# Ya no necesitara calcular gradientes para hacer inferencia
for param in net.parameters():
    param.requires_grad = False

# Calcule el la predicción de su modelo y el ground-truth
y_preds, y_tests = get_preds_tests_nn(net, train_loader)
print(classification_report(y_tests, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00         7

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### Evaluación (0.3 pts.)

Ahora probarán el funcionamiento de su clasificador con un conjunto de test.  Habiendo entrenado su clasificador, clasifiquen los documentos del conjunto de prueba `test_set` usando la función `get_preds_tests_nn`.

In [ ]:
y_preds, y_tests = get_preds_tests_nn(net, test_loader)
print(classification_report(y_tests, y_preds))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.83      0.91         6
           2       1.00      1.00      1.00         3

    accuracy                           0.92        13
   macro avg       0.93      0.94      0.93        13
weighted avg       0.94      0.92      0.92        13



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Comenten sus resultados. Estudien que ocurre para al menos tres combinaciones de `(dim_hidden_input, dim_hidden_output)`.

```
Podemos ver que cambiar la cantidad de neuronas en la capa oculta no afecta mucho la precisión del modelo pero si lo rápido que converge. A mayor cantidad de neuronas, el modelo converge más rápido.
```

In [ ]:
#Comenten sus resultados. Estudien que ocurre para al menos tres #combinaciones de `(dim_hidden_input, dim_hidden_output)`.

def entrenar(dim_hidden_input, dim_hidden_output):

    params = {
        "dim_vocab": len(train_loader.dataset.bow_cols),
        "num_classes": 3,
        "dim_hidden_input": dim_hidden_input,
        "dim_hidden_output": dim_hidden_output,
        "learning_rate": 0.4,
        "epochs": 15
    }

    # Inicialice su red neuronal
    net = MyNeuralNetwork(
        dim_vocab=params["dim_vocab"],
        num_classes=params["num_classes"],
        dim_hidden_input=params["dim_hidden_input"],
        dim_hidden_output=params["dim_hidden_output"]).cuda()

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Definir la Loss = Cross-entropy
    criterion = nn.CrossEntropyLoss().cuda()

    # Definir el optimizador = SGD: Stochastic-gradient Descent
    opti = optim.SGD(net.parameters(), lr = params["learning_rate"])

    # Definir el numero de epocas de entrenamiento
    epochs = params["epochs"]

    ## Implementar desde aqui el ciclo de entrenamiento
    ## para cada epoca en el conjunto de train
    for epoch in range(epochs):
        for (xs_bow, labels) in train_loader:

            opti.zero_grad()

            xs_bow, labels = xs_bow.to(device), labels.to(device)

            logits = net(xs_bow)

            loss = criterion(logits, labels)

            loss.backward()

            opti.step()

        total_loss = get_loss(net, train_loader, criterion)
        y_preds, y_tests = get_preds_tests_nn(net, train_loader)
        acc = (y_preds == y_tests).sum() / y_preds.shape[0]

        print("Epoca {} completada! Loss: {} Accuracy: {}".format(epoch, total_loss, acc))

    return net



L = [5, 10, 15, 20, 25]

for dim_hidden_input in L:
    entrenar(dim_hidden_input, dim_hidden_input)
    print("")
    y_preds, y_tests = get_preds_tests_nn(net, test_loader)
    print(classification_report(y_tests, y_preds))